In [33]:
import math
import pandas as pd
from itertools import chain, combinations
from numpy import linspace

In [34]:
def euklides_measure(vec1, vec2):
    result = 0
    if len(vec1) != len(vec2):
        return 0
    for i in range(len(vec1)):
        result = result + (vec1[i]-vec2[i])**2
    result = math.sqrt(result)

    return result

In [35]:
def get_csv(x):

    df = pd.read_csv(x)
    leng = len(df.columns)
    return df, leng

In [36]:
def write_to_sim_vec(sim_vec, ix, emeasure, vec_of_vecs):
    sim_vec[ix] = emeasure
    vec_of_vecs.append(sim_vec)

    return vec_of_vecs


In [37]:
def compare(test, which_line, agent, a_index, dec_atr):
    test = test_file.iloc[which_line]

    #zliczac do zmiennej liczbe wartosci atrybutu decyzyjnego -> podawana w zmiennej
    vec_of_vecs = []

    train_vec = [None] * (a_index - 1)
    test_vec = [None] * (a_index - 1)

    #tworzenie wektora odległości bezwzględnej
    for index, row in agent.iterrows():
        sim_vec = [-1 for x in range(len(dec_atr))]

        for i in range (len(train_vec)):
            train_vec[i] = row[agent.columns[i]]
            test_vec[i] = test[agent.columns[i]]

        emeasure = euklides_measure(train_vec,test_vec)


        for atr in dec_atr:

            if row[agent.columns[a_index-1]] == atr:
                vec_of_vecs = write_to_sim_vec(sim_vec, dec_atr.index(atr), emeasure, vec_of_vecs)


    return vec_of_vecs


In [38]:
def find_dec_atr(files):
    dec_atr = []
    for file in files:
        [agent, a_index] = get_csv(file)
        for index, row in agent.iterrows():
            dec = row[agent.columns[a_index-1]]
            if dec not in dec_atr:
                dec_atr.append(dec)

    return dec_atr

In [39]:
def make_vector(files, test_file, which_line, num_of_dec):
    all_vecs = []
    for file in files:
        [agent, a_index] = get_csv(file)
        all_vecs.append(compare(test_file, which_line, agent, a_index, num_of_dec))
    return all_vecs



In [40]:
def find_the_closest(k, list_v, dec_atr, agent_num):

    final_vec = [[[0 for x in range(len(dec_atr))] for x in range(k)] for x in range(agent_num)]
    ixs_to_delete = [[0 for x in range(len(dec_atr))] for x in range(agent_num)]

    save = False

    for i in range(k):
        for j in range(len(list_v)):
            for l in range(len(list_v[j])):
                for m in range(len(list_v[j][l])):
                    if list_v[j][l][m] != -1:
                        final_vec, save = try_val(final_vec, j, i, list_v[j][l], m)
                        if save:
                            ixs_to_delete[j][m] = l
    # deletion:
    # sorting the inner list to avoid moving important index down

        for i in range(len(ixs_to_delete)):
            for j in sorted(ixs_to_delete[i], reverse= True):
                del list_v[i][j]

    correction_vec = [[0 for x in range(len(dec_atr))]for x in range(agent_num)]
    for i in range(agent_num):
        for m in range(len(list_v[i])):
            for j in range(len(dec_atr)):
                if final_vec[i][k-1][j] == list_v[i][m][j]:
                    correction_vec[i][j] += 1

    return final_vec, correction_vec

In [41]:
def try_val(final_vec, agent_in_final, k_in_final, vec, m):

    fin = final_vec[agent_in_final][k_in_final]
    save = False

    for i in range(len(fin)):

        if i == m:
            if fin[m] == 0:
                fin[m] = vec[m]
                save = True

            elif fin[m] > vec[m]:
                fin[m] = vec[m]
                save = True

    return final_vec, save


In [43]:
def unify_k_vecs_new(vec, agent_num, dec_num, correction):
    result_vec = [[0 for x in range(dec_num)] for x in range(agent_num)]
    for agent in range(len(vec)):
        for k in range(len(vec[agent])):
            for dec in range(len(vec[agent][k])):
                result_vec[agent][dec] += vec[agent][k][dec]

                if k == len(vec[agent])-1 and correction[agent][dec] != 0:
                    for additional in range(correction[agent][dec]):
                        result_vec[agent][dec] += vec[agent][k][dec]


    #dzielenie przez liczbe k
    for i in range(len(result_vec)):
        for j in range(len(result_vec[i])):
            result_vec[i][j] /= (len(vec[i]) + correction[i][j])

    return result_vec

In [44]:
def change_to_relative_new(vec):
    for i in range(len(vec)):
        sum = 0
        for j in range(len(vec[i])):
            sum += vec[i][j]
        for j in range(len(vec[i])):
            vec[i][j] = 1 - vec[i][j]/sum
        sum = 0
        for j in range(len(vec[i])):
            sum += vec[i][j]
        for j in range(len(vec[i])):
            vec[i][j] = vec[i][j]/sum

    return vec


In [46]:
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

In [47]:
#q od 0 do 1 - wartosc procentowa, po osiagnieciu ktorej decyzja jest przelosowana
def Shapley_Shubik(vec, q, dec_atr):

    sums = [0 for x in range(len(dec_atr))]
    #na początku szukamy list dla poszczególnych decyzji, zapisujemy ją (listę) jako temp_vec
    for i in range(len(dec_atr)):

        #temp_vec jest pomocniczy, aby obliczyc ostateczną siłę agenta: [dec1a1, dec1a2, dec1a3..]
        temp_vec = [0 for x in range(len(vec))]

        #uzupelnienie wektora
        for m in range(len(vec)):
            temp_vec[m] = vec[m][i]

        #wyciągamy pierszy element tabeli i później odkładamy go na górę (koniec)
        agent_forces = []
        for l in range(len(temp_vec)):

            x = temp_vec.pop(0)
            forces= []

            subsets = list(powerset(temp_vec))
            sum_to_Shapley = 0
            div = 0

            #liczenie wartości sił agentów dla poszczególnych decyzji
            for sub in subsets:
                if sub:
                    sum = 0
                    for s in sub:
                        sum += s
                    if sum >= q:
                        pass
                    elif sum + x >= q:
                        sum_to_Shapley += (math.factorial(len(sub)) * math.factorial((len(temp_vec)+1)-len(sub)-1))

                else:
                    if x > q:
                        sum_to_Shapley += (math.factorial(len(sub)) * math.factorial((len(temp_vec)+1)-len(sub)-1))

            temp_vec.append(x)
            agent_forces.append(sum_to_Shapley/math.factorial(len(temp_vec)))
        sums[i] = agent_forces

            # liczenie sumy wartości sił jako siły agenta
    final_indexes = [0 for x in range(len(vec))]
    for i in range(len(sums)):
        for j in range(len(sums[i])):
            final_indexes[j] += sums[i][j]

            # mnozenie wektorow przez siłę
    for i in range(len(vec)):
        for j in range(len(vec[i])):
            vec[i][j] *= final_indexes[i]

            # sumowanie przemnozonych wektorów
    final_vec = [0 for x in range(len(dec_atr))]

    for i in range(len(dec_atr)):
        for j in range(len(vec)):
            final_vec[i] += vec[j][i]

            #wybieranie decyzji
    max_val = max(final_vec)
    decision = [dec_atr[i] for i,j in enumerate(final_vec) if j==max_val]
    return decision


In [48]:
files = ['d1.csv', 'd2.csv', 'd3.csv', 'd4.csv', 'd5.csv', 'd6.csv', 'd7.csv']
test_file, t_index = get_csv('test_w.csv')
dec_atr = find_dec_atr(files)
ageent_num = len(files)

#parametry testu
space = list(linspace(0.1,0.9,10))
zgadza = 0
length = 0

for i in range(len(test_file)):
    line_to_compare = i

    list_v = make_vector(files, test_file, line_to_compare, dec_atr)

    k_closest, correction = find_the_closest(3, list_v, dec_atr, ageent_num)

    # print("k_closest: ", k_closest)
    # print("correction ", correction)

    result = unify_k_vecs_new(k_closest, ageent_num, len(dec_atr), correction)

    # print(result)

    relative = change_to_relative_new(result)

    # print(relative)

    f_list = Shapley_Shubik(relative, 0.6, dec_atr)

    # print(test_file.iloc[line_to_compare][t_index - 1], " <- poprawna | odgadnieta -> ", f_list)

    if test_file.iloc[line_to_compare][t_index - 1] in f_list and len(f_list) == 1:
        zgadza += 1

    length += len(f_list)

print("Dokładnosc wynosi: ", zgadza/len(test_file))
print("Srednia ilosc podejmowanych decyzji: ", length/len(test_file))

    # Dla k = 3 i q = 0.6
    # Skuteczność: 0.7283464566929134
    # Srednia ilosc podejmowanych decyzji:  1.0

    #Poprawione:
    # Gdy odległość jest równa, bierzemy wszystkie równe wartości (uwzględnione oczywiście również przy jednoczeniu wektoró)
    # Podjęte decyzje to lista, w wypadku niejednoznacznej decyzji (wszystkie wartości max wchodzą).
    # Naprawiona miara Euklidesowa! Dawniej - gdy 0 to przydzielałem 1e-21, teraz zostaje 0, co DIAMETRALNIE wpływa na wyniki!
    # Drobne zmiany optymalizacyjne - czas skrócony do 1.8(3) sekundy [z 2,5s]

    # Do dodania:
    # Indeks Banzhafa

Dokładnosc wynosi:  0.7283464566929134
Srednia ilosc podejmowanych decyzji:  1.0
